<a href="https://colab.research.google.com/github/JamesBolt22/Supervised_Contrastive_learning_for_onset_detection/blob/main/Create_training_and_test_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some code has been taken from https://github.com/rohitma38/cnn-onset-detection and adapted for use in this project.

In [ ]:
#@title Import Libraries
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math
import h5py
import keras.backend as K
import glob

In [ ]:
#@title Load all data files
datadir='/content/drive/MyDrive/Processed_data_2/'
songlist=np.loadtxt('/content/drive/MyDrive/SongNameList.txt',dtype=str)
labels = np.load('/content/labels_master_2.npy', allow_pickle=True).item()
weights = np.load('/content/drive/MyDrive/Final_data/weights_master.npy', allow_pickle=True).item()
all_data_dict = np.load('/content/drive/MyDrive/Final_data/all_data_dict.npy', allow_pickle=True).item()

#Only load partition data if created
#partition = np.load('/content/drive/MyDrive/data_fold_0/split.npy', allow_pickle=True).item()

with h5py.File('/content/drive/MyDrive/Final_data/all_data.h5', 'r') as hf:
    all_data = hf["input_data"][:]

In [ ]:
#@title Split data set
partition = {'train':[], 'validation':[], 'test':[]}

#define the network inputs which are in val_split and which are in test_split
val_split = np.loadtxt("/content/drive/MyDrive/onsets_splits/onsets/splits/8-fold_cv_random_5.fold",dtype='str')
test_split = np.loadtxt("/content/drive/MyDrive/onsets_splits/onsets/splits/8-fold_cv_random_6.fold",dtype='str')

#runs through each song
for song in songlist:
  print(song)

  #produces a list of inputs files
  ids = glob.glob(datadir+song+'/*.pt.npy')

  #adds file to correct partition
  if song in val_split:
    partition['validation'].extend(ids)
  if song in test_split:
    partition['test'].extend(ids)
  else:
    partition['train'].extend(ids)

In [ ]:
#@title Save split
np.save("/content/drive/MyDrive/data_fold_7/split", partition)

In [ ]:
#@title Create training data from split

#sets up the data arrays
train_data = np.empty((len(partition['train']), 80,15,3))
train_label = np.empty((len(partition['train']), 1))

#runs through the train partition
for i in range(len(partition['train'])):

      # Select sample
      ID = partition['train'][i]

      #Load data and get label
      idx = all_data_dict[ID[:-4]]
      img = all_data[idx]
      label = labels[ID[:-4]]

      #add data to relevant array
      train_data[i] = img
      train_label[i] = label

      print(i/len(partition['train'])*100)

In [ ]:
#@title Create validation data from split

#same process as above
val_data = np.empty((len(partition['validation']), 80,15,3))
val_label = np.empty((len(partition['validation']), 1))

for i in range(len(partition['validation'])):

      # Select sample
      ID = partition['validation'][i]

    # Load data and get label
      idx = all_data_dict[ID[:-4]]
      #img = all_data[idx]
      
      label = labels[ID[:-4]]
      #val_data[i] = img
      val_label[i] = label
      print(i/len(partition['validation'])*100)
    

In [ ]:
#@title Create weights from data split

#same process as above
train_weights = np.empty((len(partition['train']),1))

for i in range(len(partition['train'])):

      # Select sample
      ID = partition['train'][i]

      # Load data and get label
      temp = weights[ID[:-4]]
      train_weights[i] = temp
      print(i/len(partition['train'])*100)

In [ ]:
#@title Save weights
weights_save = "/content/drive/MyDrive/data_fold_0/weights_data_0.020.h5"

with h5py.File(weights_save, 'w') as hf:
    hf.create_dataset("input_data",  data=train_weights)
    hf.close

In [ ]:
#@title Save train and val data
val_save = "/content/drive/MyDrive/data_fold_0/val_data.h5"
val_label_save = "/content/drive/MyDrive/data_fold_0/val_label_2.h5"
train_data_save = "/content/drive/MyDrive/Sythn_data/created_data/train_data.h5"
train_labels_save = "/content/drive/MyDrive/data_fold_0/train_labels_0.020.h5"

with h5py.File(val_save, 'w') as hf:
    hf.create_dataset("input_data",  data=val_data)
    hf.close

with h5py.File(val_label_save, 'w') as hf:
    hf.create_dataset("input_data",  data=val_label)
    hf.close

with h5py.File(train_data_save, 'w') as hf:
    hf.create_dataset("input_data",  data=train_data)
    hf.close

with h5py.File(train_labels_save, 'w') as hf:
    hf.create_dataset("input_data",  data=train_label)
    hf.close